In [1]:
import os
import cv2
import torch
import numpy as np
import mediapipe as mp
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from torchvision import models, transforms
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA


In [2]:
# 设置路径和参数
train_dir = 'ImgNationalGalleryOfArt'
image_size = (64, 64)
batch_size = 32

In [3]:
# 1. 载入数据
def load_images(train_dir, image_size=(64, 64)):
    image_paths = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.endswith('.jpg')]
    images = []
    for path in image_paths:
        img = load_img(path, target_size=image_size)
        img = img_to_array(img) / 255.0  # 归一化
        images.append(img)
    return np.array(images), image_paths


In [4]:
# 2. 目标检测（YOLOv5 检测人物）
def detect_person(image):
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    results = model(image)
    results.render()
    return results

In [5]:
# 3. 姿态估计（Mediapipe）
def extract_pose(image):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    if results.pose_landmarks:
        return np.array([[lm.x, lm.y] for lm in results.pose_landmarks.landmark])
    return None


In [6]:
# 4. CNN 特征提取（ResNet50）
def extract_features(image):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_resized = preprocess(image).unsqueeze(0)  
    resnet_model = models.resnet50(pretrained=True)
    resnet_model.eval()
    with torch.no_grad():
        features = resnet_model(img_resized)
    return features.numpy().flatten()


In [7]:
# 5. 计算相似度（Cosine Similarity）
def compute_similarity(features1, features2):
    return 1 - cosine(features1, features2)


In [8]:
# 示例
train_dir = 'ImgNationalGalleryOfArt'
images, paths = load_images(train_dir)

# 取两张图进行比较
image1 = cv2.imread(paths[0])
image2 = cv2.imread(paths[1])

# 提取特征
feat1 = extract_features(image1)
feat2 = extract_features(image2)

# 计算相似度
similarity = compute_similarity(feat1, feat2)
print(f"Cosine Similarity: {similarity}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Cosine Similarity: 0.7652250671273935


In [10]:
# # 设置路径和参数
# train_dir = 'ImgNationalGalleryOfArt'
# image_size = (64, 64)
# batch_size = 32

# # 获取所有图片路径
# image_paths = [os.path.join(train_dir, fname) for fname in os.listdir(train_dir) if fname.endswith('.jpg')]

# # 手动加载图片并转成数组
# images = []
# for path in image_paths:
#     img = load_img(path, target_size=image_size)  # 调整图片大小
#     img = img_to_array(img) / 255.0  # 转为数组并归一化
#     images.append(img)

# # 转换为 NumPy 数组
# images = np.array(images)

# # 使用 flow 方法创建生成器
# datagen = ImageDataGenerator(
#     rotation_range=30, # 随机旋转
#     width_shift_range=0.2, # 水平平移
#     height_shift_range=0.2, # 垂直平移
#     shear_range=0.2, # 随机错切变换
#     zoom_range=0.2, # 随机缩放
#     horizontal_flip=True # 随机水平翻转
# )

# train_generator = datagen.flow(
#     images, # 数据集路径
#     batch_size=batch_size,  # 批量大小
#     shuffle=True, # 是否打乱数据，通常无监督学习需要打乱
#     #class_mode=None        # 无监督学习不需要标签
# )

# print(f'已加载图片数量: {len(images)}')
